# 02. Preprocessing

Notebook này sẽ sử dụng các hàm từ `src/data_processing.py` để tiền xử lý dữ liệu.


In [6]:
import numpy as np
import sys
sys.path.append('../src')

from data_processing import (
    load_csv_data,
    build_feature_matrix,
    normalize_features,
    add_bias,
    stratified_split,
    save_processed
)


### Giải thích tạo features
Các bước xử lý và tạo đặc trưng:

1. Numeric gốc:
   - city_development_index (CDI)
   - training_hours
   - experience (chuẩn hoá: '>20'→21, '<1'→0.5, 'never'→0; missing -> median)
   - city (label encode do số lượng lớn)
2. Categorical chọn lọc (one-hot): relevent_experience, enrolled_university, education_level, last_new_job, company_size (missing -> 'Missing').
3. Interaction: CDI * Experience.
4. Polynomial: CDI^2, Experience^2.
5. Ratio: training_hours / (experience+1), CDI / (experience+1).
6. Log transform: log1p(Experience), log1p(Training_hours).
7. Chuẩn hoá toàn bộ ma trận feature (không tính bias) bằng z-score (mean/std).
8. Thêm bias term (cột toàn 1) ở đầu.
9. Stratified split 80/20 theo target để giữ tỷ lệ lớp.
10. Lưu X_train, y_train, X_test, y_test vào thư mục processed.

## 1. Xử lý dữ liệu

In [14]:
# 1. Xử lý dữ liệu (feature extraction + engineering + normalize + bias)
TRAIN_PATH = '../data/raw/aug_train.csv'
raw_data, header = load_csv_data(TRAIN_PATH)
X_raw, y, meta = build_feature_matrix(raw_data, header)
X_norm, norm_stats = normalize_features(X_raw, method='standard')
X_final = add_bias(X_norm)

print("SAU KHI XỬ LÝ & ENGINEER")
print(f"Tổng mẫu: {X_final.shape[0]:,}")
print(f"Số features (kể cả bias): {X_final.shape[1]}")
print(f"  - Base: {meta['n_base_features']}")
print(f"  - Engineered: {meta['n_engineered_features']}")
print(f"  - Categorical (one-hot): {meta['n_categorical_features']}")
print(f"Phân phối target: {np.bincount(y)}")
print(f"Median experience impute: {meta['experience_median']:.2f}")


SAU KHI XỬ LÝ & ENGINEER
Tổng mẫu: 19,158
Số features (kể cả bias): 40
  - Base: 4
  - Engineered: 7
  - Categorical (one-hot): 28
Phân phối target: [14381  4777]
Median experience impute: 9.00


## 2. Chia train / test 

In [ ]:
# 2. Chia tập train / test (stratified)
split_dict = stratified_split(X_final, y, test_size=0.2, seed=42)
X_train = split_dict['X_train']
y_train = split_dict['y_train']
X_test  = split_dict['X_test']
y_test  = split_dict['y_test']

print("STRATIFIED SPLIT 80/20")
print(f"Train: {X_train.shape[0]:,} | Target dist: {np.bincount(y_train)}")
print(f"Test : {X_test.shape[0]:,} | Target dist: {np.bincount(y_test)}")


STRATIFIED SPLIT 80/20
Train: 15,327 | Target dist: [11505  3822]
Test : 3,831 | Target dist: [2876  955]


## 3. Lưu dữ liệu đã tiền xử lý

In [ ]:
# 3. Lưu dữ liệu
save_processed('../data/processed', X_train, y_train, X_test, y_test)

print("ĐÃ LƯU: X_train.npy, y_train.npy, X_test.npy, y_test.npy\n")

print("TỔNG KẾT CUỐI CÙNG")

print(f"Tổng mẫu ban đầu : {X_final.shape[0]:,}")
print(f"Train samples    : {X_train.shape[0]:,} | Target dist: {np.bincount(y_train)}")
print(f"Test samples     : {X_test.shape[0]:,} | Target dist: {np.bincount(y_test)}")
print(f"Số features (bias + tất cả): {X_final.shape[1]}")
print(f"  - Base         : {meta['n_base_features']}")
print(f"  - Engineered   : {meta['n_engineered_features']}")
print(f"  - Categorical  : {meta['n_categorical_features']}")

ĐÃ LƯU: X_train.npy, y_train.npy, X_test.npy, y_test.npy

TỔNG KẾT CUỐI CÙNG
Tổng mẫu ban đầu : 19,158
Train samples    : 15,327 | Target dist: [11505  3822]
Test samples     : 3,831 | Target dist: [2876  955]
Số features (bias + tất cả): 40
  - Base         : 4
  - Engineered   : 7
  - Categorical  : 28
